# Tech Challenge Fase 3 - EDA
## Pós-graduação em Machine Learning
**Autor**: Alan Alves  
**Data**: 26/10/2025

In [4]:
# Carrega Módulos
import pandas as pd
from pathlib import Path
from datetime import datetime
import sys
import numpy as np
from scipy import stats

In [5]:
# Configurações
PROJECT_ROOT = Path.cwd().parent
DATA_DIR = PROJECT_ROOT / "data"
print(f"Endereço raiz: {PROJECT_ROOT}")

Endereço raiz: /Users/alan/Data Science Projects/TechChallenge3


In [6]:
# Carrega módulos 
sys.path.append(str(Path.cwd().parent))
from src.utils import glimpse

### Inspeção Inicial

In [7]:
# Load data
airlines_path = DATA_DIR / "airlines.csv"
airport_path = DATA_DIR / "airports.csv"
flights_path = DATA_DIR / "flights.csv"

start_time = datetime.now()
airlines_df = pd.read_csv(airlines_path)
airports_df = pd.read_csv(airport_path)
flights_df = pd.read_csv(flights_path, dtype={
        'SCHEDULED_DEPARTURE': str,  
        'DEPARTURE_TIME': str
    })
end_time = datetime.now()

print(f"Dados carregados com sucesso em {((end_time-start_time).seconds)} seconds.")


/var/folders/6x/tz4_svyx1mv9mc68z3jl0_s00000gn/T/ipykernel_2978/3460113801.py:9: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  flights_df = pd.read_csv(flights_path, dtype={


Dados carregados com sucesso em 38 seconds.


In [8]:
print("Airlines dataframe:")
glimpse(airlines_df)
print("\nAirports dataframe:")
glimpse(airports_df)
print("\nFlights dataframe:")
glimpse(flights_df)

Airlines dataframe:
Rows: 14
Columns: 2
           Null Count  Dtype   First Values
           ----------  -----   -------------
IATA_CODE  0           object  [UA, AA, US, F9, B6]
AIRLINE    0           object  [United Air Lines Inc., American Airlines Inc., US Airways Inc., Frontier Airlines Inc., JetBlue Airways]

Airports dataframe:
Rows: 322
Columns: 7
           Null Count  Dtype    First Values
           ----------  -----    -------------
IATA_CODE  0           object   [ABE, ABI, ABQ, ABR, ABY]
AIRPORT    0           object   [Lehigh Valley International Airport, Abilene Regional Airport, Albuquerque International Sunport, Aberdeen Regional Airport, Southwest Georgia Regional Airport]
CITY       0           object   [Allentown, Abilene, Albuquerque, Aberdeen, Albany]
STATE      0           object   [PA, TX, NM, SD, GA]
COUNTRY    0           object   [USA, USA, USA, USA, USA]
LATITUDE   3           float64  [40.65236, 32.41132, 35.04022, 45.44906, 31.53552]
LONGITUDE  3       

                     Null Count  Dtype    First Values
                     ----------  -----    -------------
YEAR                 0           int64    [2015, 2015, 2015, 2015, 2015]
MONTH                0           int64    [1, 1, 1, 1, 1]
DAY                  0           int64    [1, 1, 1, 1, 1]
DAY_OF_WEEK          0           int64    [4, 4, 4, 4, 4]
AIRLINE              0           object   [AS, AA, US, AA, AS]
FLIGHT_NUMBER        0           int64    [98, 2336, 840, 258, 135]
TAIL_NUMBER          14721       object   [N407AS, N3KUAA, N171US, N3HYAA, N527AS]
ORIGIN_AIRPORT       0           object   [ANC, LAX, SFO, LAX, SEA]
DESTINATION_AIRPORT  0           object   [SEA, PBI, CLT, MIA, ANC]
SCHEDULED_DEPARTURE  0           object   [0005, 0010, 0020, 0020, 0025]
DEPARTURE_TIME       86153       object   [2354, 0002, 0018, 0015, 0024]
DEPARTURE_DELAY      86153       float64  [-11.0, -8.0, -2.0, -5.0, -1.0]
TAXI_OUT             89047       float64  [21.0, 12.0, 16.0, 15.0, 11.0]

### Preparação Inicial dos Dados  

Os dados contém três tabelas:
* `flihgts_df` (voos): é a principal tabel a e contém informação detalhada de cada voo. 
* `airlines_df`: traz o nomes da empresas de aviações.
* `airports_df`  (aeroportos): traz informações sobre os aeroportos como por exemplo cidade, país etc.

**Obs.:** Existe varias colunas com falta de dados e algumas delas faltam dados para a maior parte das observações. 

O próximo passo é fazer a junção dessas três tabelas.

In [9]:
# --- Preparação Inicial - junção das três tabelas ---

# Função auxiliar para renomear colunas
def make_airport_columns_names(prefix):
    """Cria um dicionário para renomear colunas de aeroporto com um prefixo."""
    new_names_dict = {
        'AIRPORT': f'{prefix}_AIRPORT_NAME', 
        'CITY': f'{prefix}_CITY',
        'STATE': f'{prefix}_STATE', 
        'COUNTRY': f'{prefix}_COUNTRY', 
        'LATITUDE': f'{prefix}_LATITUDE',
        'LONGITUDE': f'{prefix}_LONGITUDE'
    }
    return new_names_dict

# Pula etapa de juntar tabelas caso o trabalha esteja sendo retomado e o processamento já havia sido realizado
df_out_path = PROJECT_ROOT / 'data/df.csv'
if df_out_path.exists():
    df = pd.read_csv(df_out_path, dtype={
        'SCHEDULED_DEPARTURE': str,  
        'DEPARTURE_TIME': str
    })
    
    # Libera memória
    del(flights_df)
    del(airlines_df)
    del(airports_df)
    
    print("Tabela com junções já realizada carregada do disco local:")
    glimpse(df)
else:
    # Junção com 'airlines_df' para obter o nome da companhia
    df = pd.merge(
        flights_df, 
        airlines_df, 
        left_on='AIRLINE', 
        right_on='IATA_CODE',
        suffixes=('_flight', '_airline'),
        how = 'left'
    )
    
    # Renomeia o nome da companhia e remove colunas redundantes
    df = df.rename(columns={'AIRLINE_airline': 'AIRLINE_NAME', 'IATA_CODE':'AIRLINE'})
    df = df.drop(columns=['AIRLINE_flight'])

    # Junção com 'airports_df' para dados de ORIGEM
    df = pd.merge(
        df, 
        airports_df, 
        left_on='ORIGIN_AIRPORT', 
        right_on='IATA_CODE',
        how = 'left'
    )
    
    # Renomeia colunas de origem e remove IATA_CODE redundante
    df = df.rename(columns=make_airport_columns_names('ORIGIN'))
    df = df.drop(columns=['IATA_CODE'])

    # Junção com 'airports_df' para dados de DESTINO
    df = pd.merge(
        df, 
        airports_df, 
        left_on='DESTINATION_AIRPORT', 
        right_on='IATA_CODE',
        how = 'left'
    )
    
    # Renomeia colunas de destino e remove IATA_CODE redundante
    df = df.rename(columns=make_airport_columns_names('DEST'))
    df = df.drop(columns=['IATA_CODE'])

    # Salva tabela
    df.to_csv(df_out_path, index=False)

    # Libera memória
    del(flights_df)
    del(airlines_df)
    del(airports_df)

    print("Associações realizada e tabela salva com sucesso para o caminho selecionado. ")
    df.info()

/var/folders/6x/tz4_svyx1mv9mc68z3jl0_s00000gn/T/ipykernel_2978/351551266.py:19: DtypeWarning: Columns (6,7,32,33,34,35,38,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(df_out_path, dtype={


Tabela com junções já realizada carregada do disco local:
Rows: 5819079
Columns: 44
                     Null Count  Dtype    First Values
                     ----------  -----    -------------
YEAR                 0           int64    [2015, 2015, 2015, 2015, 2015]
MONTH                0           int64    [1, 1, 1, 1, 1]
DAY                  0           int64    [1, 1, 1, 1, 1]
DAY_OF_WEEK          0           int64    [4, 4, 4, 4, 4]
FLIGHT_NUMBER        0           int64    [98, 2336, 840, 258, 135]
TAIL_NUMBER          14721       object   [N407AS, N3KUAA, N171US, N3HYAA, N527AS]
ORIGIN_AIRPORT       0           object   [ANC, LAX, SFO, LAX, SEA]
DESTINATION_AIRPORT  0           object   [SEA, PBI, CLT, MIA, ANC]
SCHEDULED_DEPARTURE  0           object   [0005, 0010, 0020, 0020, 0025]
DEPARTURE_TIME       86153       object   [2354, 0002, 0018, 0015, 0024]
DEPARTURE_DELAY      86153       float64  [-11.0, -8.0, -2.0, -5.0, -1.0]
TAXI_OUT             89047       float64  [21.0, 12

### Formatando Colunas de Tempo 
As colunas `SCHEDULED_DEPARTURE` e `DEPARTURE_TIME` estão no formato HHMM e para ser util é necessário separar em duas colunas contendo a hora e o minuto e converte-las para número. Outras colunas também apresentam esse mesmo formato, mas será alterada apeas se avaliarmos ser útil para análise mais a frente. 

In [10]:
# Obtem a data em uma única coluna
df['DATE'] = pd.to_datetime(df[['YEAR', 'MONTH', 'DAY']])

# Obtem hora e minutos em coluna separada
df['SCHEDULED_DEPARTURE_HOUR'] = df['SCHEDULED_DEPARTURE'].str[:2]
df['SCHEDULED_DEPARTURE_MIN'] = df['SCHEDULED_DEPARTURE'].str[2:4]
df['DEPARTURE_HOUR'] = df['DEPARTURE_TIME'].str[:2]
df['DEPARTURE_MIN'] = df['DEPARTURE_TIME'].str[2:4]

# Converte para número
df['SCHEDULED_DEPARTURE_HOUR'] = pd.to_numeric(df['SCHEDULED_DEPARTURE_HOUR'], errors='coerce').astype('Int64')
df['SCHEDULED_DEPARTURE_MIN'] = pd.to_numeric(df['SCHEDULED_DEPARTURE_MIN'], errors='coerce').astype('Int64')
df['DEPARTURE_HOUR'] = pd.to_numeric(df['DEPARTURE_HOUR'], errors='coerce').astype('Int64')
df['DEPARTURE_MIN'] = pd.to_numeric(df['DEPARTURE_MIN'], errors='coerce').astype('Int64')



### Sumário Descritivo das Colunas

In [11]:
round(df.describe().T,2)

,count,mean,min,25%,50%,75%,max,std
YEAR,5819079.0,2015.0,2015.0,2015.0,2015.0,2015.0,2015.0,0.0
MONTH,5819079.0,6.524085,1.0,4.0,7.0,9.0,12.0,3.405137
DAY,5819079.0,15.704594,1.0,8.0,16.0,23.0,31.0,8.783425
DAY_OF_WEEK,5819079.0,3.926941,1.0,2.0,4.0,6.0,7.0,1.988845
FLIGHT_NUMBER,5819079.0,2173.092742,1.0,730.0,1690.0,3230.0,9855.0,1757.063999
DEPARTURE_DELAY,5732926.0,9.370158,-82.0,-5.0,-2.0,7.0,1988.0,37.080942
TAXI_OUT,5730032.0,16.071662,1.0,11.0,14.0,19.0,225.0,8.895574
WHEELS_OFF,5730032.0,1357.170841,1.0,935.0,1343.0,1754.0,2400.0,498.009356
SCHEDULED_TIME,5819073.0,141.685892,18.0,85.0,123.0,173.0,718.0,75.210582
ELAPSED_TIME,5714008.0,137.006189,14.0,82.0,118.0,168.0,766.0,74.211072


### Contagem de Dados Ausentes por Coluna

In [12]:
print(df.isna().sum())

YEAR                              0
MONTH                             0
DAY                               0
DAY_OF_WEEK                       0
FLIGHT_NUMBER                     0
TAIL_NUMBER                   14721
ORIGIN_AIRPORT                    0
DESTINATION_AIRPORT               0
SCHEDULED_DEPARTURE               0
DEPARTURE_TIME                86153
DEPARTURE_DELAY               86153
TAXI_OUT                      89047
WHEELS_OFF                    89047
SCHEDULED_TIME                    6
ELAPSED_TIME                 105071
AIR_TIME                     105071
DISTANCE                          0
WHEELS_ON                     92513
TAXI_IN                       92513
SCHEDULED_ARRIVAL                 0
ARRIVAL_TIME                  92513
ARRIVAL_DELAY                105071
DIVERTED                          0
CANCELLED                         0
CANCELLATION_REASON         5729195
AIR_SYSTEM_DELAY            4755640
SECURITY_DELAY              4755640
AIRLINE_DELAY               

### Análise de Estatística Descritiva e Tratamento de Dados Ausentes

1. **Dataset**:
    * 5,8M de registros, suficiente para treinar qualquer modelo de Machine Learning.
    * Escopo Temporal: Todos os registros são do ano de 2015 (min e max da coluna year igual a 2015). Sazonalidade ao longo do ano pode ser capturada, mas não será possível calcular sozonalidade ou mudança anual. **Devido não ser possível avaliar a tendência ano-a-ano, cuidados devem ser tomados ao colocar o modelo em produção**, pois a realidade atual pode ser diferente de 10 anos atrás. 

2. **Variaveis resposta: DEPARTURE_DELAY e ARRIVAL_DELAY**

    * *ARRIVAL_DELAY*:
        * voos chegam um pouco atrasados - média de 4.4 minutos. 
        * Metade dos voos chegam pelo menos 5 minutos adiantados - mediana de -5min
        * Atrasos extremos (máxima de 1971 min) fazem a média de voos serem atrasados em 4.4min. Contudo, mediana -5min, indica distribuição assimétrica com a maioria dos voos *não atrasando na chegada* e poucos outros voos com atrasos curtos e *outros em menores quantidades, com atrasos extremos*
        * O devio padrão é quase 9 vezes maior que a média, indicando dados espalhados, dificeis de serem previstos. O projeto de classificação (ATRASO vs NÃO ATRASO), terá maior probabilidade de sucesso.

    * *DEPARTURE_DELAY*:
        * Média de 9.37 minutos, e mediana de -2 minutos. 50% dos voos saem no horário ou adiantados, mas a menor quantidade de voos que saem atrasados, puxa a média para cima, implicando em uma distribuição assimétrica. 

    * **Implicação para Modelagem**:
        * *Variável alvo*: talvez, mais importante que saber se o voo irá sair atrasado é saber se chegará atrasado. Como a média da variável `DEPARTURE_DELARY`é mais que o dobro da variavel `ARRIVAL_DELAY` é provavel que nem todo voo que sai atrasado chega atrasado. Portanto, **a variavel alvo a ser trabalhada é `ARRIVAL_DELAY`**. 
        
        * *ATRASADO*: para muitas aplicações, atrasos de 5min é suficiente para desencadear uma série de situações indesejadas. Tendo isso em mente, uma nova váriavel, `IS_DELAYED` será criada. Essa nova variável **indica se um voo chegou atrasado em seu destino, por 5min ou mais**.
        
        * *Classe desbalanceada*: definir o limit para atraso como 5min, significa puxar mais à direita (destro da distribuição) o limite, indicando um desbalaceamento entre as classes, que impactará diretamente na modelagem.

3. **Dados Ausentes**
    * Voos Concluídos: As colunas ARRIVAL_DELAY e ELAPSED_TIME têm 5.714.008 registros, enquanto os dados tem 5.819.079 registros. A diferença (~104k) provavelmente corresponde aos voos CANCELLED (cancelado - 1.5% do total, ~89k) e DIVERTED (desviado, 0.26% do total, ~15k). A coluna `MEAN` indica a fração de voos de cancelados e desviados. Portanto a quantidade de observações disponível para modelagem de voos atrasado é **5.714.008**.
        * **Tratamento**: excluir observações com dados ausentes para `ELAPSED_TIME`. O mesmo se aplica a outras colunas relacionadas, como por exemplo `TAXI_IN`, `TAXI_OUT`, `WHEELS_ON`, `WHEELS_OFF`, `AIR_TIME`, `DEPARTURE_DELAY`, `DEPARTURE_TIME` e `ARRIVAL_TIME`. Dados para essas conlunas só existirão para voos concluidos. Portanto podem seguramente serem excluidos da base de dados onde o objetivo do modelo é prever atraso. 

    * *CANCELLATION_REASON*: os valores NA existem para todo voo não cancelado (~98,5%) dos voos. 
        - **Tratamento**: Excluir a coluna. A coluna é irrelevante, pois o modelo será treinado apenas sobre voos concluídos.
    * Colunas de atraso: colunas que finalizam com o sufixo **DELAY** são colunas que indicam razões para atrasos. Apenas 1.063.439, são preenchidas. Hipótese: Elas só são preenchidas quando há atraso . 
        - **Tratamento**: Preencher com 0, assumindo que NaN significa "0 minutos de atraso" daquele tipo.

    * *Latitude/Longitude*: Quase 500k voos estão sem dados de ORIGIN_LATITUDE ou DEST_LATITUDE. Hipótese: é possível que o arquivo airports.csv não contenha todos os IATA_CODEs presentes nos voos, talvez para aeroportos com menores ou regionais, talvez aeroportos menores ou regionais. O mesmo se aplica à outras colunas relacionadas aos dados do aeroporto, como `CITY`, `COUNTRY`e outras. 
        - **Tratamento**: para análises geográficas ou que dependem da localização, essas linhas serão removidas. Para modelagem, novas variáveis será criadas a partir dela, como por exemplo, `REGIONAL_AIRPORT`, sendo verdadeiro para dados ausentes. Pra as colunas com dados relacionados ao aeropoto, será preenchido com `MISSING`. 

    * *TAIL_NUMBER*: Número de registro da aeronava. 14721 com dados ausentes. Possivelmente algumas aeronaves não tem registro. A priori, não parece ser relevante para análise.
        - **Tratamento**: será preenchido com "LACKING REGISTRATION NUMBER".
  

4. **Variáveis Explicatórias**
    * *TAXI_OUT e TAXI_IN*:  O tempo médio de TAXI_OUT (16.07 min) é mais que o dobro do TAXI_IN (7.43 min). Possivelmente o tempo de *taxiamento* para decolagem influencia mais os atrasos (assumindo que a coluna ARRIVAL_TIME é indica o tempo após o TAXI_IN) que o *taxiamento* pós pouso. A variável `TAXI_IN`, não fará parte da próxima etapa de EDA e nem da modelagem, pois é evento que ocorre imediatamente antes da conlusão do voo e seu efeito prático é nulo. 

    * *LATE_AIRCRAFT_DELAY* tem a maior média (23.47 min - calculado apenas entre os voos atrasados) entre os tipos de atraso. Isso sugere que um voo atrasado é um forte preditor de que o próximo voo daquela aeronave também atrasará.

    * *SECURITY_DELAY* (média 0.076 min) é praticamente irrelevante na maioria dos casos. 75% dos voos (75% percentil) com atraso detalhado tiveram 0 minutos de atraso de segurança.

    * *Horários*: As colunas de `HOUR` e `MIN` criadas estão bem distribuídas (média e mediana centralizadas). Podem ser utilizada com segurança para verificar se existe padrão de atraso a depender da hora do dia, por exemplo.

    * *DISTANCE*: Os voos variam de muito curtos (21 milhas) a muito longos (4983 milhas), com a maioria sendo de curta a média distância (mediana de 647 milhas).

## Modelagem supervisonada - Classificação e Problemática

Para efeitos práticos, o modelo servirá como solução para o problema de **perda de conexões devido à atraso nos voos**.
Algumas conexões são curtas (da ordem de 25 a 30) e atraso de 5min pode implicar na perda da conexão por um passageiro. Portanto, ser capaz de prever se um voo irá atrasar por 5min ou mais, com pelo menos 30min de atecedencia, pode ajudar na **gestão proativa de conexões de passageiros**. Por exemplo, outros voos podem serem reservados para passageiros em voos que há previsão de atraso amenizando o decontentamento do cliente. 

**Objetivo**: prever se um voo irá se atrasar por 5min ou mais, com 30min de atecedencia à chegada agendada do voo. 

### Engenharia de Features (Feature Engineering)

A criação de features será feita antes da limpeza de dados ausentes, para aproveitar informações de dados ausentes.

1. **Definição da Variável Alvo (Target)**

    * IS_DELAYED_5MIN:
        * 1 (Verdadeiro): Se o voo teve um atraso na chegada (ARRIVAL_DELAY) de 5 minutos ou mais.
        * 0 (Falso): Se o voo chegou no horário ou adiantado (atraso < 5 minutos).

Voos com dados de chegada ausentes (ex: CANCELLED == 1) receberão o valor NaN, pois seu resultado é desconhecido. Esses voos serão excluidos antes da modelagem.

2. **Features de Contexto e Tempo**

    - SCHEDULED_DEPARTURE_HOUR: A hora "cheia" da partida (ex: 13h). O tráfego aéreo varia dependendo da hora do dia.
    - IS_WEEKEND: Uma flag booleana (1 ou 0) que indica se o dia da semana (DAY_OF_WEEK) é Sábado (6) ou Domingo (7). Dias da semana é esperado maior tráfego.

3. **Features de Risco Histórico da Rota**
A hipótese é que o desempenho passado de uma rota é um forte preditor de seu desempenho futuro.
Para capturar isso, seguimos dois passos:
    * Definição da Rota (ROUTE_ID): Primeiro, definimos uma "rota" de forma específica, combinando: ORIGIN_AIRPORT, DESTINATION_AIRPORT, AIRLINE, SCHEDULED_DEPARTURE_HOUR
    Aqui, parte-se do principio que voos de mesmo origem e destino, mas em horas diferente ou/e de companhias diferente podem ter atrasos ou pontualidades diferentes.
    * Cálculo das Estatísticas
        * ROUTE_AVG_ARR_DELAY: O atraso médio histórico de chegada para esta rota.
        * ROUTE_CANCEL_RATE: A taxa (0.0 a 1.0) de cancelamento desta rota.
        * ROUTE_DIVERT_RATE: A taxa de desvio desta rota.
        * ROUTE_ON_TIME_RATE: A taxa de voos que chegaram pontualmente (atraso < 5 min) nesta rota.

4. **Features de Ausência de Dados (Missingness)**
    IS_REGIONAL_AIRPORT: Conforme a hipótese levantada na análise exploratória, criamos uma flag (1 ou 0). Ela é 1 (Verdadeiro) se o voo não possui dados de latitude/longitude na origem OU no destino. Isso pode indicar que o aeroporto é menor (talvez!).

In [13]:
import pandas as pd
import numpy as np

def run_feature_engineering(df):
    """
    Executa a etapa de engenharia de features.
    
    Assume que 'df' já contém os dados unidos de flights, airlines e airports.
    """
    
    print("--- Iniciando Engenharia de Features ---")
    
    # ---
    # 1. Criação de Features de Base (Contexto)
    # ---
    
    # Garante que 'SCHEDULED_DEPARTURE_HOUR' exista.
    # Se 'SCHEDULED_DEPARTURE' for 1345 (13:45), isso extrai 13.
    if 'SCHEDULED_DEPARTURE_HOUR' not in df.columns:
        print("Criando 'SCHEDULED_DEPARTURE_HOUR' a partir de 'SCHEDULED_DEPARTURE'...")
        df['SCHEDULED_DEPARTURE_HOUR'] = df['SCHEDULED_DEPARTURE'].astype(str).str[:2]
    
    # Feature é fim de semana
    df['IS_WEEKEND'] = df['DAY_OF_WEEK'].isin([6, 7]).astype(int)
    
    print("...Features de Base criadas.")

    # ---
    # 2. Criação da Variável Alvo (Target)
    # ---
    # Alvo: 1 (True) se o atraso na CHEGADA for de 5 minutos ou mais. 0 (False) caso contrário.
    
    df['IS_DELAYED_5MIN'] = np.where(
        df['ARRIVAL_DELAY'].isna(), 
        np.nan,  # Mantém NaN se o atraso for NaN
        (df['ARRIVAL_DELAY'] >= 5).astype(int)
    )
    
    print("...Variável Alvo 'IS_DELAYED_5MIN' criada.")

    # ---
    # 3. Criação do ID da Rota (ROUTE_ID)
    # ---
    # Definição: Origem + Destino + Companhia + Hora Agendada de Partida
    
    print("...Criando 'ROUTE_ID'.")
    df = df.reset_index(drop=True)
    df['ROUTE_ID'] = df['ORIGIN_AIRPORT'].astype(str) + '_' + \
                     df['DESTINATION_AIRPORT'].astype(str) + '_' + \
                     df['AIRLINE'].astype(str) + '_' + \
                     df['SCHEDULED_DEPARTURE_HOUR'].astype(int).astype(str)
    
    print(f"...'ROUTE_ID' criado. (Ex: {df['ROUTE_ID'].iloc[0]})")

    # ---
    # 4. Criação de Features de Risco da Rota (Histórico)
    # ---
 
    print("...Calculando ROUTE_AVG_ARR_DELAY (Média de Atraso da Rota)...")
    df['ROUTE_AVG_ARR_DELAY'] = df.groupby('ROUTE_ID')['ARRIVAL_DELAY'].transform('mean')

    print("...Calculando ROUTE_CANCEL_RATE (Taxa de Cancelamento da Rota)...")
    df['ROUTE_CANCEL_RATE'] = df.groupby('ROUTE_ID')['CANCELLED'].transform('mean')

    print("...Calculando ROUTE_DIVERT_RATE (Taxa de Desvio da Rota)...")
    df['ROUTE_DIVERT_RATE'] = df.groupby('ROUTE_ID')['DIVERTED'].transform('mean')
    
    print("...Calculando ROUTE_ON_TIME_RATE (Taxa de Pontualidade da Rota)...")
    # Para esta feature, precisamos de uma coluna auxiliar 'IS_ON_TIME'
    # Consideramos 'pontual' (1) se o atraso for < 5 min.
    # Voos cancelados (NaN) não são pontuais (0).
    df['temp_IS_ON_TIME'] = np.where(
        df['ARRIVAL_DELAY'].isna(), 
        0, # Se for NaN, não foi pontual
        (df['ARRIVAL_DELAY'] < 5).astype(int)
    )
    df['ROUTE_ON_TIME_RATE'] = df.groupby('ROUTE_ID')['temp_IS_ON_TIME'].transform('mean')
    df = df.drop(columns=['temp_IS_ON_TIME'])
    
    print("...Features de Risco da Rota criadas.")

    # ---
    # 5. Criação de Features de Ausência
    # ---
    
    print("...Criando 'IS_REGIONAL_AIRPORT'.")
    # IS_REGIONAL_AIRPORT: True (1) se dados de lat/lon de origem OU destino estiverem faltando
    df['IS_ORIGIN_REGIONAL_AIRPORT'] = df['ORIGIN_LATITUDE'].isna().astype(int)
    df['IS_DEST_REGIONAL_AIRPORT'] = df['DEST_LATITUDE'].isna().astype(int)

    print("--- Engenharia de Features Concluída ---")
    
    return df

df = run_feature_engineering(df)
print('\n')
glimpse(df)

--- Iniciando Engenharia de Features ---
...Features de Base criadas.
...Variável Alvo 'IS_DELAYED_5MIN' criada.
...Criando 'ROUTE_ID'.
...'ROUTE_ID' criado. (Ex: ANC_SEA_AS_0)
...Calculando ROUTE_AVG_ARR_DELAY (Média de Atraso da Rota)...
...Calculando ROUTE_CANCEL_RATE (Taxa de Cancelamento da Rota)...
...Calculando ROUTE_DIVERT_RATE (Taxa de Desvio da Rota)...
...Calculando ROUTE_ON_TIME_RATE (Taxa de Pontualidade da Rota)...
...Features de Risco da Rota criadas.
...Criando 'IS_REGIONAL_AIRPORT'.
--- Engenharia de Features Concluída ---


Rows: 5819079
Columns: 58
                            Null Count  Dtype           First Values
                            ----------  -----           -------------
YEAR                        0           int64           [2015, 2015, 2015, 2015, 2015]
MONTH                       0           int64           [1, 1, 1, 1, 1]
DAY                         0           int64           [1, 1, 1, 1, 1]
DAY_OF_WEEK                 0           int64           

### Tratamento de Dados Ausentes

Os dados ausentes serão tratados conforme política definada acima.


In [14]:
df_cleaned = df.copy()

print(f"Linhas originais: {df_cleaned.shape[0]}")

# ---
# 1. Tratamento de Voos Concluídos (Remoção de Cancelados/Desviados)
# ---
# Remove linhas onde 'ELAPSED_TIME' é NaN.
# Isso remove os voos cancelados e desviados,
# limpando também os NaN em colunas como ARRIVAL_DELAY, TAXI_IN, etc.

print("\nEtapa 1: Removendo voos não concluídos...")
linhas_antes = df_cleaned.shape[0]
df_cleaned = df_cleaned.dropna(subset=['ELAPSED_TIME'])
linhas_removidas = linhas_antes - df_cleaned.shape[0]
print(f"Etapa 1: {linhas_removidas} linhas (voos não concluídos) foram removidas.")
print(f"Linhas restantes: {df_cleaned.shape[0]}")


# ---
# 2. Tratamento das Colunas de Razão de Atraso (Suffix _DELAY)
# ---
# Preenche NaN com 0 para as colunas de *razões* de atraso.

# Identifica  as colunas de razão de atraso
# excluindo as duas colunas principais (['ARRIVAL_DELAY', 'DEPARTURE_DELAY'])
main_delay_cols = ['ARRIVAL_DELAY', 'DEPARTURE_DELAY']
delay_reason_cols = [col for col in df_cleaned.columns 
                     if col.endswith('_DELAY') and col not in main_delay_cols]

print(f"\nEtapa 2: Preenchendo NaN com 0 para as colunas: {delay_reason_cols}")
df_cleaned[delay_reason_cols] = df_cleaned[delay_reason_cols].fillna(0)


# ---
# 3. Tratamento de TAIL_NUMBER
# ---
# Preenche o número de registro ausente com uma string específica.

print("\nEtapa 4: Preenchendo NaN em 'TAIL_NUMBER'...")
df_cleaned['TAIL_NUMBER'] = df_cleaned['TAIL_NUMBER'].fillna('LACKING REGISTRATION NUMBER')

#---
# 4. Tratametno das Colunas relacionados ao aeropoto
# ---
print("\nEtapa 5: Preenchendo NAN nas colunas relacionados à localização do aeroporto...")
_origin_columns = ['ORIGIN_AIRPORT_NAME','ORIGIN_CITY','ORIGIN_STATE','ORIGIN_COUNTRY']
_dest_columns = ['DEST_AIRPORT_NAME','DEST_CITY','DEST_STATE','DEST_COUNTRY']
_columns = _origin_columns + _dest_columns
df_cleaned[_columns] = df_cleaned[_columns].fillna('MISSING')

# ---
# 5. Tratamento de CANCELLATION_REASON
# ---
# Preenche o número de registro ausente com uma string específica.

print("\nEtapa 5: Removendo coluna 'CANCELLATION_REASON'...")
df_cleaned = df_cleaned.drop(axis=1, columns=['CANCELLATION_REASON'])


# Verificação Final
# ---
print("\n--- Verificação Final de Dados Ausentes ---")
# O código abaixo mostrará a contagem de NaNs restantes no dataframe
print(df_cleaned.isna().sum())

print("\nPrimeiras linhas da tabela final:")
glimpse(df_cleaned)

# Salva arquivo limpo
# Força conversão de todas as colunas tipo 'object' para string para evitar falha ao salvar parquet
try:
    for col in df_cleaned.select_dtypes(include='object').columns:
        df_cleaned[col] = df_cleaned[col].astype(str) 
    df_cleaned.to_parquet(str(PROJECT_ROOT/'data/df_limpo_com_features.parquet'), index=False)
    print("\nDados limpos com features criadas salvo com sucesso em data/df_limpo_com_features.parquet")
except Exception as e:
    print("\nErro ao salvar arquivo: ",e)

Linhas originais: 5819079

Etapa 1: Removendo voos não concluídos...
Etapa 1: 105071 linhas (voos não concluídos) foram removidas.
Linhas restantes: 5714008

Etapa 2: Preenchendo NaN com 0 para as colunas: ['AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY', 'ROUTE_AVG_ARR_DELAY']

Etapa 4: Preenchendo NaN em 'TAIL_NUMBER'...

Etapa 5: Preenchendo NAN nas colunas relacionados à localização do aeroporto...

Etapa 5: Removendo coluna 'CANCELLATION_REASON'...

--- Verificação Final de Dados Ausentes ---
YEAR                               0
MONTH                              0
DAY                                0
DAY_OF_WEEK                        0
FLIGHT_NUMBER                      0
TAIL_NUMBER                        0
ORIGIN_AIRPORT                     0
DESTINATION_AIRPORT                0
SCHEDULED_DEPARTURE                0
DEPARTURE_TIME                     0
DEPARTURE_DELAY                    0
TAXI_OUT                           0
WHEELS_